In [ ]:
import sounddevice as sd
import queue
import json
from vosk import Model, KaldiRecognizer



In [ ]:
# -------- CONFIG --------
MODEL_PATH = "/Users/olixstudios/Documents/workspace/Projects/vosk-demo/models/vosk-model-small-en-us-0.15"  # change if needed
SAMPLE_RATE = 16000
# ------------------------


def main():
    print("Loading model...")
    model = Model(MODEL_PATH)
    recognizer = KaldiRecognizer(model, SAMPLE_RATE)

    audio_queue = queue.Queue()

    def audio_callback(indata, frames, time, status):
        if status:
            print("Status:", status)
    
        # DEBUG: show that audio bytes are arriving
        # print("Audio chunk size:", len(indata))
    
        audio_queue.put(bytes(indata))

    print("Starting microphone... Speak into the mic. Ctrl+C to stop.")
    with sd.RawInputStream(
        samplerate=SAMPLE_RATE,
        blocksize=8000,
        dtype="int16",
        channels=1,
        callback=audio_callback,
        device=4,
    ):
        while True:
            data = audio_queue.get()
            if recognizer.AcceptWaveform(data):
                result = recognizer.Result()
                text = json.loads(result).get("text", "")
                if text.strip():
                    print("Final:", text)
            else:
                # Partial results while you're speaking
                partial = recognizer.PartialResult()
                partial_text = json.loads(partial).get("partial", "")
                if partial_text.strip():
                    print("Partial:", partial_text, end="\r")



In [ ]:
def audio_callback(indata, frames, time, status):
    if status:
        print("Status:", status)

    # DEBUG: show that audio bytes are arriving
    print("Audio chunk size:", len(indata))

    audio_queue.put(bytes(indata))

In [ ]:
main()